In [1]:
import ee
try:
    ee.Initialize()
except: 
    ee.Authenticate()
    ee.Initialize()

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [2]:
import xarray as xr

ds = xr.open_zarr('/mnt/hdd1/fran/fran_sep2018_clean.zarr')
ds


<xarray.Dataset> Size: 65GB
Dimensions:      (wl: 202, x: 16310, y: 9885)
Coordinates:
  * wl           (wl) float64 2kB 0.4066 0.4134 0.4201 ... 1.983 1.989 1.996
  * x            (x) float64 130kB 3.14e+05 3.14e+05 ... 3.629e+05 3.629e+05
  * y            (y) float64 79kB 6.266e+06 6.266e+06 ... 6.237e+06 6.237e+06
Data variables:
    reflectance  (wl, x, y) int16 65GB dask.array<chunksize=(202, 10, 1000), meta=np.ndarray>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import itertools
from matplotlib.patches import Rectangle

In [ ]:
# rename vars and set crs
ds_gcs = ds_gcs.rename({"X": "x", "Y": "y"})

with urllib.request.urlopen(
    "https://storage.googleapis.com/fran-share/wl_dict.json"
) as url:
    wldict = json.load(url)

dsx = ds_gcs.rename(wldict)
# get new band names
wl_bands = list(dsx.data_vars)
# convert to dim
dsx = dsx[wl_bands].to_array(dim="wl")
dsx = dsx.sortby("wl")

# drop bad bands
dsx = dsx.where(
    ((dsx.wl > 0.400) & (dsx.wl < 1.340))
    | ((dsx.wl > 1.455) & (dsx.wl < 1.790))
    | ((dsx.wl > 1.955) & (dsx.wl < 2.400)),
    drop=True,
)

# write crs
dsx.rio.write_crs("EPSG:32734", inplace=True)
# bb = minx, miny, maxx, maxy
bb = [313988, 6236797, 362919, 6266456]
dsx = dsx.rio.clip_box(*bb)

dx = riox.open_rasterio(
    "https://storage.googleapis.com/fran-share/SA_NLC_2018_GEO.tif", chunks="auto"
)

# convert to crs of template
dsxb = dsx.rio.transform_bounds(
    "+init=epsg:4326",
    densify_pts=21,
)

# crop using bounding box
dx = dx.rio.clip_box(*dsxb)
dx = dx.rio.reproject_match(dsx)
dx = dx.stack(z=("x", "y")).squeeze(drop=True)

# encode labels
dx.data = LabelEncoder().fit_transform(dx.data)
dx = dx.unstack("z")

# Combine the two datasets
dx = dx.assign_coords(
    {
        "x": dsx.x,
        "y": dsx.y,
    }
)
dx = dx.to_dataset(name="label")
dsx = dsx.fillna(0).to_dataset(name="reflectance")
ds_join = dsx.drop("spatial_ref").merge(dx.drop("spatial_ref"))
ds_join = ds_join.chunk({"x": 1000, "y": 1000})

# write
ds_join.to_zarr(
    "gcs://fran-share/fran_torch.zarr",
    consolidated=True,
    storage_options={"project": "science-sharing", "token": "anon"},
)

In [ ]:
# visualise a chunk
# plot spectra for a point